In [12]:
from langgraph.graph import START, END, StateGraph, MessagesState
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os

print(" All imports successful")

 All imports successful


In [13]:
# Load environment variables
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

if not openai_api_key:
    raise ValueError("OPENAI_API_KEY not found! Please set it in your .env file.")

print(" API key loaded successfully")

 API key loaded successfully


In [26]:
# Ensure env is reloaded if needed (no additional key checks here)
load_dotenv(override=True)
openai_api_key = os.getenv("OPENAI_API_KEY")
print("OPENAI_API_KEY present:", bool(openai_api_key))

OPENAI_API_KEY present: True


In [27]:
# Initialize LLM
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,  # Lower temperature for more precise tool usage
    api_key=openai_api_key
)

print(f"LLM initialized: {llm}")

LLM initialized: profile={'max_input_tokens': 128000, 'max_output_tokens': 16384, 'image_inputs': True, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True, 'structured_output': True, 'image_url_inputs': True, 'pdf_inputs': True, 'pdf_tool_message': True, 'image_tool_message': True, 'tool_choice': True} client=<openai.resources.chat.completions.completions.Completions object at 0x0000026EE3C1B230> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000026EE3C1B5C0> root_client=<openai.OpenAI object at 0x0000026EE3C1A060> root_async_client=<openai.AsyncOpenAI object at 0x0000026EE3C1B360> model_name='gpt-4o-mini' temperature=0.0 model_kwargs={} openai_api_key=SecretStr('**********') stream_usage=True


In [28]:
# Define the system prompt for customer support
SYSTEM_PROMPT = """You are a helpful and empathetic customer support representative for TechCorp Electronics.

Your responsibilities:
- Listen carefully to customer issues and concerns
- Remember what the customer has told you about their problem
- Provide helpful troubleshooting steps
- Offer solutions based on the context of the conversation
- Be professional but friendly

Always:
1. Acknowledge the customer's issue with empathy
2. Reference previous details from the conversation when relevant
3. Ask clarifying questions if needed
4. Provide clear, step-by-step solutions
5. Offer to escalate if the problem is beyond your scope

Remember: You have access to the full conversation history - use it to provide personalized support!"""

print(" System prompt defined")

 System prompt defined


In [29]:
# Initialize the LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)

# Create the agent node function
def customer_support_agent(state: MessagesState) -> dict:
    """
    The main agent node that processes messages and maintains context.
    
    Args:
        state: MessagesState containing the conversation history
        
    Returns:
        Updated state with the agent's response added to messages
    """
    # Prepare messages: system prompt first, then conversation history
    messages = [SystemMessage(content=SYSTEM_PROMPT)] + state["messages"]
    
    # Get response from LLM
    response = llm.invoke(messages)
    
    # Return the updated state with the new assistant message
    return {"messages": [response]}

print(" Agent function created")

 Agent function created


In [30]:
# Create the StateGraph
graph_builder = StateGraph(MessagesState)

# Add the agent node
graph_builder.add_node("agent", customer_support_agent)

# Add edges to connect START -> agent -> END
graph_builder.add_edge(START, "agent")
graph_builder.add_edge("agent", END)

print(" StateGraph structure created")

 StateGraph structure created


In [31]:
# Initialize the checkpointer for memory persistence
checkpointer = MemorySaver()

# Compile the graph with the checkpointer
customer_support_graph = graph_builder.compile(checkpointer=checkpointer)

print(" Graph compiled with MemorySaver checkpointer")
print("   Conversation memory is now persistent!")

 Graph compiled with MemorySaver checkpointer
   Conversation memory is now persistent!


In [32]:
# Helper function to manage conversations
def run_support_conversation(user_input: str, thread_id: str = "support-chat-1"):
    """
    Run a single turn in the customer support conversation.
    
    Args:
        user_input: The user's message
        thread_id: Unique ID for this conversation thread
        
    Returns:
        The agent's response as a string
    """
    # Create the input state with user's message
    input_state = {"messages": [HumanMessage(content=user_input)]}
    
    # Invoke the graph with thread_id for memory persistence
    output = customer_support_graph.invoke(
        input_state,
        config={"configurable": {"thread_id": thread_id}}
    )
    
    # Extract the agent's response
    agent_response = output["messages"][-1]
    return agent_response.content

print(" Helper function created")

 Helper function created


In [40]:
# Interactive Multi-Turn Conversation
import uuid

print("=" * 80)
print(" CUSTOMER SUPPORT CHATBOT - INTERACTIVE MODE")
print("=" * 80)

# Create a unique thread for this session
thread_id = f"session-{str(uuid.uuid4())[:8]}"
print(f"\nThread ID: {thread_id}")
print("Type 'exit' to end the conversation\n")
print("-" * 80 + "\n")

turn = 1
while True:
    # Get user input
    user_input = input(f"👤 You (Turn {turn}): ").strip()
    
    # Check for exit command
    if user_input.lower() == "exit":
        print("\n Conversation ended. Thank you for chatting!")
        break
    
    # Skip empty messages
    if not user_input:
        print(" Please enter a message.\n")
        continue
    
    # Get agent response
    agent_response = run_support_conversation(user_input, thread_id)
    print(f"\n Agent (Turn {turn}): {agent_response}\n")
    print("-" * 80 + "\n")
    
    turn += 1

 CUSTOMER SUPPORT CHATBOT - INTERACTIVE MODE

Thread ID: session-5bba2a19
Type 'exit' to end the conversation

--------------------------------------------------------------------------------


 Agent (Turn 1): I completely understand your desire for a durable system that will stand the test of time. To help you better, could you please share what specific type of system you’re looking for? Are you interested in a laptop, desktop, gaming system, or perhaps another type of electronic device? Additionally, if you have any particular requirements or budget in mind, that would be helpful too!

--------------------------------------------------------------------------------


 Agent (Turn 2): Thank you for clarifying that you're looking for a laptop! For a laptop that lasts longer, here are a few key factors to consider:

1. **Build Quality:** Look for laptops with aluminum or magnesium chassis, as they tend to be more durable than plastic ones.

2. **Battery Life:** Opt for laptops that of

In [37]:
# Verify the conversation was stored in memory
final_conversation_state = customer_support_graph.get_state(
    config={"configurable": {"thread_id": thread_id}}
)

print("\n" + "=" * 80)
print(" CONVERSATION MEMORY VERIFICATION")
print("=" * 80)
print(f"\nTotal messages in thread '{thread_id}': {len(final_conversation_state.values['messages'])}\n")

for i, message in enumerate(final_conversation_state.values["messages"], 1):
    message_type = type(message).__name__
    preview = message.content[:70] + "..." if len(message.content) > 70 else message.content
    print(f"[{i}] {message_type:12} | {preview}")

print("\n" + "=" * 80)
print(" Memory verification complete!")


 CONVERSATION MEMORY VERIFICATION

Total messages in thread 'session-bb8f10dd': 4

[1] HumanMessage | tell me about dell laptop
[2] AIMessage    | Could you please specify what type of information you are looking for ...
[3] HumanMessage | what system can i buy for that will last longer
[4] AIMessage    | I understand that you're looking for a laptop that will have longevity...

 Memory verification complete!


In [39]:
# Optional: Test thread isolation with a demo
print("\n" + "=" * 80)
print(" THREAD ISOLATION DEMO")
print("=" * 80)
print("\nEach customer can have their own conversation thread without interference.")
print(f"\nExample: Your thread '{thread_id}' is isolated from any other customer's thread.")
print("\nTo start a new conversation with a different customer, simply create a new thread_id!")
print("\nThis enables:")
print("   Multiple simultaneous customer chats")
print("   Conversation history isolation")
print("   Production-ready scalability")
print("\n" + "=" * 80)


 THREAD ISOLATION DEMO

Each customer can have their own conversation thread without interference.

Example: Your thread 'session-bb8f10dd' is isolated from any other customer's thread.

To start a new conversation with a different customer, simply create a new thread_id!

This enables:
   Multiple simultaneous customer chats
   Conversation history isolation
   Production-ready scalability

